In [1]:

import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from pathlib import Path

# Define user data directory for session persistence
USER_DIR = Path("user_data").absolute()
if not USER_DIR.exists():
    USER_DIR.mkdir()

profile = "marketplace"
CHROMEDRIVER_PATH = Path("chromedriver/chromedriver.exe").absolute()

In [2]:
def create_driver(profile_name):
    """Creates a WebDriver instance for a given profile name using absolute paths."""
    try:
        options = uc.ChromeOptions()
        profile_path = USER_DIR / profile_name
        options.add_argument(f"--user-data-dir={str(profile_path)}") # Use saved session
        options.add_argument("--start-maximized")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-extensions")
        options.add_argument("--dns-prefetch-disable")
        options.add_argument("--disable-gpu")
        # 🔽 Disable pop-up blocking 🔽
        options.add_argument("--disable-popup-blocking")

        driver = uc.Chrome(
            options=options,
            driver_executable_path=str(CHROMEDRIVER_PATH) # Use string format for path
        )
        driver.maximize_window()
        driver.set_page_load_timeout(30)
        return driver
    except Exception as e:
        print(e)
        return None

In [3]:
def manual_login_setup(profile_name):
    """Sets up the driver and allows the user to manually log in."""
    driver = create_driver(profile_name=profile_name)
    if driver is not None:
        print("Please log in manually in the opened browser window.")
        print("Once logged in, you can close the browser or leave it open.")
        print("After login, press Enter in the console to continue or just close the browser if only login is needed.")
        input("Press Enter to continue after login (or just close browser for login only)...")
        print("Login process completed or skipped.")
        return driver
    else:
        print("Failed to create driver instance for manual login.")
        return None

In [4]:
def marketplace_scraper_step_by_step(driver, product_name="used iphone"): # Modified function name
    """Scrapes Facebook Marketplace for the given product name, step-by-step for debugging."""
    if driver is None:
        print("Driver is not initialized. Please ensure manual login setup is done correctly.")
        return

    try:
        # Convert product name to URL-friendly format
        search_query = product_name.replace(" ", "%20")
        marketplace_url = f"https://www.facebook.com/marketplace/search/?query={search_query}"

        driver.get("https://www.facebook.com/marketplace")
        time.sleep(5)  # Wait for the page to load
        print("✅ Marketplace page loaded!")

        # 🔹 Search for product
        search_box = driver.find_element(By.XPATH, '//input[@placeholder="Search Marketplace"]')
        search_box.send_keys(product_name) # Use product_name parameter here
        search_box.send_keys(Keys.RETURN)
        time.sleep(5)  # Wait for search results to load
        print("✅ Search completed!")

        # 🔹 Open first 1 listing (for debugging)
        listings = driver.find_elements(By.XPATH, '//div[contains(@style, "max-width") and contains(@style, "min-width")]//a[contains(@href, "/marketplace/item/")]')[:1]  # Just one listing for now
        print(f"Found {len(listings)} listings based on structure (containing max-width and min-width in style).") # Debug print
        if not listings:
            print("No listings found using structure-based XPath (containing max-width and min-width in style). Check HTML structure for changes.")
            return  # Exit if no listings found
        print(listings)

        if listings: # Proceed only if listings are found
            listing_url = listings[0].get_attribute("href")
            print("Opening listing URL:", listing_url)
            driver.execute_script("window.open(arguments[0]);", listing_url)
            time.sleep(5)
            driver.switch_to.window(driver.window_handles[1]) # Switch to new tab
            print("✅ Switched to listing tab.")

            try:
                # --- DEBUGGING SECTION - Focus here ---
                print("🔍 Attempting to find 'Send' button...")
                send_message_button = driver.find_element(By.XPATH, '//div[@aria-label="Send" and @role="button"]')
                # go to its child div
                child_message_button = send_message_button.find_element(By.XPATH, ".//div") # Find the first child div
                print("Child div of 'Send' button found:", child_message_button)

                # You can further inspect the child element:
                print("Child div tag name:", child_message_button.tag_name)
                print("Child div text (if any):", child_message_button.text)
                print("Child div attributes:", child_message_button.get_attribute('outerHTML'))

                if child_message_button: # Keep the original click logic for now
                    print("Clicked 'Send' button on listing page.")
                    driver.execute_script("arguments[0].click();", send_message_button)
                    child_message_button.click()
                    print("✅ 'Send' button clicked.")
                else:
                    print("⚠️ 'Send' button element is found but might be considered 'False' for some reason.")

                time.sleep(2) # Delay after clicking Send button
                print("⏳ Waited 2 seconds after clicking 'Send'.")

                # ---  (Rest of the messaging logic - can add later if needed) ---
                # chat_box = WebDriverWait(driver, 10).until(
                #     EC.presence_of_element_located((By.XPATH, "//div[@aria-label='Message']"))
                # )
                # chat_box.send_keys("Hi! I'm interested in your item. Is it still available? 😊")
                # chat_box.send_keys("\n")
                # print("Message sent successfully!")


            except Exception as e:
                print(f"❌ Error interacting with seller in listing tab: {e}")
                print(e) # Print the full error

            # Keep tab open for inspection in notebook
            # driver.close() # Commented out close for debugging
            # driver.switch_to.window(driver.window_handles[0]) # Commented out switch back

    except Exception as e:
        print("❌ Error during Marketplace search:", e)
        print(e) # Print full error

In [5]:
driver = manual_login_setup(profile_name=profile)
if driver:
    print("Driver initialized successfully after manual login.")
else:
    print("Driver initialization failed.")

Please log in manually in the opened browser window.
Once logged in, you can close the browser or leave it open.
After login, press Enter in the console to continue or just close the browser if only login is needed.
Login process completed or skipped.
Driver initialized successfully after manual login.


In [6]:
if driver: # Only run if driver was initialized
    marketplace_scraper_step_by_step(driver, product_name="used iphone") # You can change product_name
else:
    print("Driver not initialized, skipping scraper.")

✅ Marketplace page loaded!
✅ Search completed!
Found 1 listings based on structure (containing max-width and min-width in style).
[<undetected_chromedriver.webelement.WebElement (session="73d8f07968fea886d68fd2c6ea7e6a92", element="f.D8B57EBF40F1CDA5A66784DA24F5C947.d.B9F789AC10A5A60BDCD1EAF559EE2067.e.49")>]
Opening listing URL: https://www.facebook.com/marketplace/item/1072736881289005/?ref=search&referral_code=null&referral_story_type=post&tracking=browse_serp%3Abfb0a14c-0469-45d0-90d6-81166c68ca63&__tn__=!%3AD
✅ Switched to listing tab.
🔍 Attempting to find 'Send' button...
Child div of 'Send' button found: <undetected_chromedriver.webelement.WebElement (session="73d8f07968fea886d68fd2c6ea7e6a92", element="f.5FB619D0BEC71D1ECB49069DA4784BA8.d.F6FB820C6428B481A6E8EF85B5DFBFE6.e.120")>
Child div tag name: div
Child div text (if any): 
Child div attributes: <div role="none" class="x1ja2u2z x78zum5 x2lah0s x1n2onr6 xl56j7k x6s0dn4 xozqiw3 x1q0g3np xi112ho x17zwfj4 x585lrc x1403ito x972